<a href="https://colab.research.google.com/github/kibutan/freeCodeCamp/blob/main/machine-learning-with-python/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [97]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.0-dev20211206


In [98]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-12-07 01:41:16--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2021-12-07 01:41:17 (11.7 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2021-12-07 01:41:17--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [100]:
train_tsv = pd.read_csv("/content/train-data.tsv",sep="\t",header = None)
test_tsv = pd.read_csv("/content/valid-data.tsv",sep="\t",header = None)
train_tsv[0] = train_tsv[0].replace(["ham","spam"],[0,1])
test_tsv[0] = test_tsv[0].replace(["ham","spam"],[0,1])
dataset_train_v = tf.data.Dataset.from_tensor_slices((train_tsv[1].values,train_tsv[0].values))
dataset_val_v = tf.data.Dataset.from_tensor_slices((test_tsv[1].values,test_tsv[0].values))


In [101]:
train_tsv

,0,1
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,0,just woke up. yeesh its late. but i didn't fal...
4175,0,what do u reckon as need 2 arrange transport i...
4176,1,free entry into our £250 weekly competition ju...
4177,1,-pls stop bootydelious (32/f) is inviting you ...


In [102]:
dataset_train_v

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [103]:
for ex in dataset_train_v.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'you can never do nothing'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'mum say we wan to go then go... then she can shun bian watch da glass exhibition...'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'never y lei... i v lazy... got wat? dat day \xc3\xbc send me da url cant work one...'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [104]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set=set()
for text_tensor , label_tensor in dataset_train_v:
  some_tokens_text = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens_text)
vocab_size = len(vocabulary_set)
vocab_size

7483

In [105]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [106]:
example_text =next(iter(dataset_train_v))[0].numpy()
print(example_text)

b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'


In [107]:
example_label =next(iter(dataset_train_v))[1].numpy()
print(example_label)

0


In [108]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[4380, 3849, 4895, 4168, 7439, 852, 1293, 5676, 367, 3239, 2862, 1875, 6738, 2184, 4945, 3239, 1309, 6811, 6738, 4042, 3843, 4604, 367, 4543, 673, 3746, 6738, 2606, 1067, 4168, 6954, 5331]


In [109]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  # encoded_label = encoder.encode(label.numpy())
  return encoded_text,label

In [110]:
dataset_train_v

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [111]:
def encode_map_fn(text,label):
  encoded_text, label = tf.py_function(encode,
                                       inp=[text,label],
                                       Tout=(tf.int64, tf.int64))
  encoded_text.set_shape([None])
  label.set_shape([])
  return encoded_text,label

all_encoded_train_data = dataset_train_v.map(encode_map_fn)
all_encoded_test_data = dataset_val_v.map(encode_map_fn)


In [112]:
tf.__version__

'2.8.0-dev20211206'

In [113]:
# BUFFER_SIZE = 50000
BATCH_SIZE = 32
# TAKE_SIZE = 5000

In [114]:
len(all_encoded_train_data)

4179

In [115]:
train_data = all_encoded_train_data.padded_batch(BATCH_SIZE)
test_data = all_encoded_test_data.padded_batch(BATCH_SIZE)

In [116]:
sample_text, sample_labels = next(iter(train_data))
sample_text[0],sample_labels[0]

(<tf.Tensor: shape=(50,), dtype=int64, numpy=
 array([4380, 3849, 4895, 4168, 7439,  852, 1293, 5676,  367, 3239, 2862,
        1875, 6738, 2184, 4945, 3239, 1309, 6811, 6738, 4042, 3843, 4604,
         367, 4543,  673, 3746, 6738, 2606, 1067, 4168, 6954, 5331,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [117]:
vocab_size += 2
#何故か2回やってSizeを7485にするとモデルFitがとおる
#無いとModel.fit時にGraph.error

[+2じゃないとModel.fit時にエラーを吐く件について](https://www.tensorflow.org/tutorials/load_data/text#:~:text=Since%20you%20use%200%20for%20padding%20and%201%20for%20out%2Dof%2Dvocabulary%20(OOV)%20tokens%2C%20the%20vocabulary%20size%20has%20increased%20by%20two%3A)

Deepl:パディングに0、OOV（Out-of-Vocabulary）トークンに1を使用しているので、語彙数が2増えています。

In [118]:
#7485がGood
vocab_size

7485

In [134]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
for units in [64,64]:
  model.add(tf.keras.layers.Dense(units,activation="relu"))
model.add(tf.keras.layers.Dense(2))
model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
model.fit(train_data, epochs=2,validation_data=test_data)

Epoch 1/2
131/131 [==============================] - 15s 84ms/step - loss: 0.2037 - accuracy: 0.9294 - val_loss: 0.0742 - val_accuracy: 0.9763
Epoch 2/2
131/131 [==============================] - 10s 79ms/step - loss: 0.0336 - accuracy: 0.9919 - val_loss: 0.0471 - val_accuracy: 0.9856


In [124]:
test_data

<PaddedBatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [254]:
def get_string_label(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch,axis = 1)
  print("pred_label_int",predicted_int_labels.numpy())
  label = "ham" if predicted_int_labels.numpy()[0] == 0 else "spam"
  return label

In [255]:
# predicted_scores_test = model.predict(predict_data)
# predicted_label_test = get_string_label(predicted_scores_test)
# print(predicted_label_test)

pred_label_int [0 1 0 1 1 0 0]
ham


In [126]:
loss,accuracy = model.evaluate(test_data)
print("loss:{:.2}, aCCURACY :{:.2%}".format(loss,accuracy))

44/44 [==============================] - 1s 27ms/step - loss: 0.0485 - accuracy: 0.9842
loss:0.048, aCCURACY :98.42%


In [217]:
# zipped_list

[('how are you doing today', 'ham'),
 ('sale today! to stop texts call 98912460324', 'spam'),
 ('i dont want to go. can we try it a different day? available sat', 'ham'),
 ('our new mobile video service is live. just install on your phone to start watching.',
  'spam'),
 ('you have won £1000 cash! call to claim your prize.', 'spam'),
 ("i'll bring it tomorrow. don't forget the milk.", 'ham'),
 ('wow, is your arm alright. that happened to me one time too', 'ham')]

In [186]:
# https://tech-branch.9999ch.com/archives/408
# ラベルとデータで別れたリストからDataFrameをつくる

zipped_list = list(zip(["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ],["ham", "spam", "ham", "spam", "spam", "ham", "ham"]))

predict_test_df = pd.DataFrame(zipped_list)
predict_test_df.replace(["ham","spam"],[-1,-1],inplace=True)

dataset_predict_test_v = tf.data.Dataset.from_tensor_slices((predict_test_df[0].values,predict_test_df[1].values))
all_encoded_predict_test = dataset_predict_test_v.map(encode_map_fn)
predict_data = all_encoded_predict_test.padded_batch(BATCH_SIZE)

In [214]:
predict_test_df

,0,1
0,how are you doing today,-1
1,sale today! to stop texts call 98912460324,-1
2,i dont want to go. can we try it a different d...,-1
3,our new mobile video service is live. just ins...,-1
4,you have won £1000 cash! call to claim your pr...,-1
5,i'll bring it tomorrow. don't forget the milk.,-1
6,"wow, is your arm alright. that happened to me ...",-1


In [146]:
model.predict(predict_data)

array([[ 3.5958166 , -2.2025204 ],
       [-0.30718696,  0.60141575],
       [ 3.6405382 , -2.2334123 ],
       [-1.7641248 ,  3.4357967 ],
       [-1.9674537 ,  3.7847724 ],
       [ 4.2650476 , -2.6397324 ],
       [ 3.2797697 , -1.9956802 ]], dtype=float32)

In [147]:
predict_test_df

,0,1
0,how are you doing today,0
1,sale today! to stop texts call 98912460324,1
2,i dont want to go. can we try it a different d...,0
3,our new mobile video service is live. just ins...,1
4,you have won £1000 cash! call to claim your pr...,1
5,i'll bring it tomorrow. don't forget the milk.,0
6,"wow, is your arm alright. that happened to me ...",0


In [ ]:
for ex in dataset_predict_test_v.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'how are you doing today'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'sale today! to stop texts call 98912460324'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'i dont want to go. can we try it a different day? available sat'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'our new mobile video service is live. just install on your phone to start watching.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'you have won \xc2\xa31000 cash! call to claim your prize.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


PredictのためにModelのエクスポートが必要
[参考](https://www.tensorflow.org/tutorials/load_data/text#export_the_model_2)

In [258]:
def predict_message(pred_text):
  pred_list = pd.DataFrame(list((zip([pred_text] ,[-1]))))
  # predict_test_df.replace(["ham","spam"],[-1,-1],inplace=True)
  print(pred_list)
  dataset_predict = tf.data.Dataset.from_tensor_slices((pred_list[0].values,pred_list[1].values))
  all_encoded_predict = dataset_predict.map(encode_map_fn)
  predict_data = all_encoded_predict.padded_batch(BATCH_SIZE)
  predicted_scores = model.predict(predict_data)
  predicted_label = get_string_label(predicted_scores)
  prediction = predicted_label
  print("return",predicted_label)
  return [pred_text,prediction]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)


                          0  1
0  how are you doing today? -1
pred_label_int [0]
return ham
['how are you doing today?', 'ham']


In [259]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    # print(msg)
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


                         0  1
0  how are you doing today -1
pred_label_int [0]
return ham
                                            0  1
0  sale today! to stop texts call 98912460324 -1
pred_label_int [1]
return spam
                                                   0  1
0  i dont want to go. can we try it a different d... -1
pred_label_int [0]
return ham
                                                   0  1
0  our new mobile video service is live. just ins... -1
pred_label_int [1]
return spam
                                                   0  1
0  you have won £1000 cash! call to claim your pr... -1
pred_label_int [1]
return spam
                                                0  1
0  i'll bring it tomorrow. don't forget the milk. -1
pred_label_int [0]
return ham
                                                   0  1
0  wow, is your arm alright. that happened to me ... -1
pred_label_int [0]
return ham
You passed the challenge. Great job!
